<img src="https://github.com/ktafakkori/feloopy/raw/main/assets/logo/logo3.png" width="2%"> **Cutting Stock Problem (CSP)**

* Copyright (c) 2022-2024, Keivan Tafakkori. All rights reserved.
* See the file LICENSE file for licensing details.

####  *Required modules*

In [1]:
import feloopy as flp

####  Dataset

In [2]:
dt = flp.data_toolkit(key=0)

items = dt.set(name="items", bound=[0,4])
rolls = dt.set(name="rolls", bound=[0, 99])
items_length = dt.store(name="items_length", value=[20, 45, 50, 55, 75])
items_demand = dt.store(name="items_demand", value=[48, 35, 24, 10, 8])
rolls_length = dt.store(name="rolls_length", value=110)

####  Exact Optimization Algorithms

In [3]:
def csp(m):
    z = m.ivar(name="z", dim=[rolls,items], bound=[0,10])
    y = m.bvar(name="y", dim=[rolls])
    m.obj(m.sum(y[roll] for roll in rolls))
    m.con([m.sum(z[roll,item] for roll in rolls) >= items_demand[item] for item in items])
    m.con([m.sum(items_length[item]*z[roll,item] for item in items) <= rolls_length for roll in rolls])
    m.con([m.sum(z[roll,item] for item in items) <= len(rolls)*y[roll] for roll in rolls])
    return m

m = flp.search(name="csp",
               environment=csp, 
               directions=["min"],
               method="exact",
               interface="cplex",
               solver="cplex",
               key_vars=["y", "z"],
               verbose=False)

m.clean_report()


√ Healthy

┌─ FelooPy v0.3.0 ───────────────────────────────────────────────── Released April 2024 ─┐
│                                                                                        │
│ Date: 2024-05-10                                                      Interface: cplex │
│ Time: 19:12:14                                                           Solver: cplex │
│ Name: csp                                                                Method: exact │
│ Type: single-objective                                                  X Unconfigured │
│                                                                                        │
└────────────────────────────────────────────────────────────────────────────────────────┘

┌─ Model ────────────────────────────────────────────────────────────────────────────────┐
│                       B       I       P       F       E       S       O       C        │
╞════════════════════════════════════════════════════════════════════════════

In [4]:
y = m.get("y")
z = m.get("z")

patterns = {} 
for roll in rolls:
    if y[roll] > 0.8:
        pattern = [z[roll, item] for item in items]
        pattern_str = "["+', '.join(map(str, pattern))+"]"
        if pattern_str in patterns:
            patterns[pattern_str] += 1
        else:
            patterns[pattern_str] = 1

for pattern, count in patterns.items():
    print(f"Pattern {pattern} is used {count} times.")

Pattern [1.0, 0.0, 0.0, 0.0, 1.0] is used 8 times.
Pattern [1.0, 2.0, 0.0, 0.0, 0.0] is used 17 times.
Pattern [3.0, 0.0, 1.0, 0.0, 0.0] is used 7 times.
Pattern [0.0, 0.0, 2.0, 0.0, 0.0] is used 8 times.
Pattern [0.0, 0.0, 0.0, 2.0, 0.0] is used 5 times.
Pattern [0.0, 1.0, 1.0, 0.0, 0.0] is used 1 times.
Pattern [2.0, 0.0, 1.0, 0.0, 0.0] is used 1 times.
